# Data Import

### Saving the last updated date

In [1]:
import os

try:
    with open('last_updated.txt', 'r') as f:
        lines = f.readlines()
        last_updated_date = lines[0].strip()

        if len(lines) > 1:
            last_updated_balance = float(lines[1].strip())
        else:
            last_updated_balance = float(input("Balance not found. Please enter the last known balance: "))        
except FileNotFoundError:
    print("File not found. Defaulting to a very old date and asking for balance.")
    last_updated_date = '2025-04-15'
    last_updated_balance = float(input("Please enter the last known balance: "))
except ValueError:
    print("Balance format invalid. Asking for balance.")
    last_updated_balance = float(input("Please enter the last known balance: "))

print(f"Last Updated Date: {last_updated_date}")
print(f"Last Updated Balance: {last_updated_balance}")


Last Updated Date: 2025-07-17
Last Updated Balance: 20302.94


## API

### Calling Splitwise API to extract groups, friends and basic details of expenses. Also converting the JSON file to dataframe

In [ ]:
import requests

API_KEY = "UNIQUE_API_KEY"
response = requests.get(
    "https://secure.splitwise.com/api/v3.0/get_expenses",
    headers={"Authorization": f"Bearer {API_KEY}"}
)

if response.status_code == 200:
    print("Success! Here are your expenses:")
    print(response.json())
else:
    print("Error:", response.status_code)
    print(response.text)

Success! Here are your expenses:
{'expenses': [{'id': 3993168383, 'group_id': 79916117, 'expense_bundle_id': None, 'description': 'Payment', 'repeats': False, 'repeat_interval': None, 'email_reminder': False, 'email_reminder_in_advance': -1, 'next_repeat': None, 'details': '', 'comments_count': 1, 'payment': True, 'creation_method': 'payment', 'transaction_method': 'offline', 'transaction_confirmed': False, 'transaction_id': None, 'transaction_status': None, 'cost': '164.02', 'currency_code': 'CAD', 'repayments': [{'from': 76291542, 'to': 35378642, 'amount': '164.02'}], 'date': '2025-08-18T18:32:50Z', 'created_at': '2025-08-18T18:32:50Z', 'created_by': {'id': 35378642, 'first_name': 'Sanjay', 'last_name': 'Kurian', 'picture': {'medium': 'https://s3.amazonaws.com/splitwise/uploads/user/default_avatars/avatar-orange4-100px.png'}, 'custom_picture': False}, 'updated_at': '2025-08-18T18:32:50Z', 'updated_by': None, 'deleted_at': None, 'deleted_by': None, 'category': {'id': 18, 'name': 'Gene

In [ ]:
import pandas as pd
import requests
from datetime import datetime
from collections import defaultdict

# 1. Set up API connection
API_KEY = "Bearer UNIQUE_API_KEY"
BASE_URL = "https://secure.splitwise.com/api/v3.0"

def get_groups():
    response = requests.get(
        f"{BASE_URL}/get_groups",
        headers={"Authorization": API_KEY}
    )
    return {g['id']: g['name'] for g in response.json().get('groups', [])}

def get_all_users():
    """Get all unique users from friends, current user, and expense repayments"""
    # Get friends
    friends_response = requests.get(
        f"{BASE_URL}/get_friends",
        headers={"Authorization": API_KEY}
    )
    friends = friends_response.json().get('friends', [])
    
    # Get current user
    current_user_response = requests.get(
        f"{BASE_URL}/get_current_user",
        headers={"Authorization": API_KEY}
    )
    current_user = current_user_response.json().get('user', {})
    
    # Get all expenses to find users in repayments
    expenses_response = requests.get(
        f"{BASE_URL}/get_expenses",
        headers={"Authorization": API_KEY},
        params={"limit": "1000"}
    )
    expenses = expenses_response.json().get('expenses', [])
    
    # Collect all unique users  
    users = defaultdict(dict)
    
    # Add friends
    for user in friends:
        user_id = user.get('id')
        users[user_id] = {
            'name': user.get('name') or f"{user.get('first_name', '')} {user.get('last_name', '')}".strip(),
            'active': True
        }
    
    # Add current user
    if current_user:
        user_id = current_user.get('id')
        users[user_id] = {
            'name': current_user.get('name') or f"{current_user.get('first_name', '')} {current_user.get('last_name', '')}".strip(),
            'active': True
        }
    
    # Add users from repayments (mark as inactive if not already found)
    for expense in expenses:
        for repayment in expense.get('repayments', []):
            for user_type in ['to_user', 'from_user']:
                user_id = repayment.get(user_type)
                if user_id and user_id not in users:
                    users[user_id] = {
                        'name': f"User {user_id} (removed)",
                        'active': False
                    }
    
    return users

def get_expenses_dataframe():
    # Get all users
    all_users = get_all_users()
    groups = get_groups()
    
    # Get expenses
    response = requests.get(
        f"{BASE_URL}/get_expenses",
        headers={"Authorization": API_KEY},
        params={"limit": "2000",
                "updated_after": last_updated_date}
    )
    expenses = response.json().get('expenses', [])
    
    # Create base columns
    base_columns = ['Date', 'Group', 'Description', 'Category', 'Cost', 'Currency']
    
    # Create user columns (active users first, then inactive)
    active_users = [u['name'] for u in all_users.values() if u['active']]
    inactive_users = [u['name'] for u in all_users.values() if not u['active']]
    user_columns = active_users + inactive_users
    
    # Process each expense
    rows = []
    for expense in expenses:
        # Basic info
        row = {
            'Date': expense.get('date'),
            'Group': groups.get(expense.get('group_id'), 'No Group'),
            'Description': expense.get('description'),
            'Category': expense.get('category', {}).get('name'),
            'Cost': expense.get('cost'),
            'Currency': expense.get('currency_code')
        }
        
        # Initialize all user amounts to 0
        row.update({name: 0.0 for name in user_columns})
        
        for user_data in expense.get('users', []):
            user_info = user_data.get('user', {})
            full_name = f"{user_info.get('first_name', '')} {user_info.get('last_name', '')}".strip()
            net_balance = float(user_data.get('net_balance', 0))
        
            # Only update if this user exists in our columns
            if full_name in user_columns:
                row[full_name] = net_balance
        
        rows.append(row)
    
    # Create DataFrame
    split_df = pd.DataFrame(rows, columns=base_columns + user_columns)
    
    # Format date
    if not split_df.empty and 'Date' in split_df:
        split_df['Date'] = pd.to_datetime(split_df['Date']).dt.date
    
    # Format numeric columns
    numeric_cols = ['Cost'] + user_columns
    split_df[numeric_cols] = split_df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0.0).round(2)
    
    return split_df

# Run and save
try:
    split_df = get_expenses_dataframe()
    print(split_df.head())
except Exception as e:
    print(f"Error occurred: {e}")

         Date            Group        Description               Category  \
0  2025-08-18  171 Anne Street            Payment                General   
1  2025-08-12  171 Anne Street  Taste of shawarma                General   
2  2025-08-11  171 Anne Street        Nawabs uber                   Taxi   
3  2025-08-01               Us            Payment                General   
4  2025-07-30  171 Anne Street                Cig  Entertainment - Other   

     Cost Currency  Aakash Raj  Preethi None  Aditya None  Rushda Najeeb  \
0  164.02      CAD         0.0           0.0          0.0           0.00   
1   32.00      CAD         0.0           0.0          0.0           0.00   
2   40.00      CAD         0.0           0.0          0.0           0.00   
3   37.32      CAD         0.0           0.0          0.0          37.32   
4   14.00      CAD         0.0           0.0          0.0           0.00   

   Beuju A  Deepak None  Aswin Narayanan  sai meghana  Praveen Kumar  \
0      0.0    

### Extracting the payment information into readable format. Also calculating the amount contributed by the payer in the transaction.

#### For Example: In an equal expense of CAD 30 which involves Aakash, Sanjay and Aditya, for which Aditya paid, Aditya contributed CAD 10.

In [4]:
split_df["Payment_or_not"] = split_df['Description']

In [5]:
name_columns = split_df.columns.difference(['Date', 'Group', 'Description', 'Category', 'Cost', 'Currency', 'Payment_or_not'])

def update_description(row):
    if row['Description'] == 'Payment':
        payee = row[name_columns][row[name_columns] < 0]
        payer = row[name_columns][row[name_columns] > 0]
        if len(payer) > 0 and len(payee) > 0:
            payee_name = payee.index[0]
            payer_name = payer.index[0]
            return f"{payer_name} paid {payee_name}"
    return row['Description']

split_df['Description'] = split_df.apply(update_description, axis=1)

def adjust_contribution(row):
    if row['Payment_or_not'] == 'Payment':
        payer = row[name_columns][row[name_columns] > 0]
        payee = row[name_columns][row[name_columns] < 0]
        if len(payer) > 0 and len(payee) > 0:
            payer_name = payer.index[0]
            row[payer_name] = -abs(float(row[payer_name]))
            payee_name = payee.index[0]
            row[payee_name] = -(float(row[payee_name]))
        return row

    else:
        payer = row[name_columns][row[name_columns] > 0]
        payee = row[name_columns][row[name_columns] < 0]
        if len(payer) > 0 and len(payee) > 0 and payer.iloc[0] == row['Cost']:
            pass
        elif len(payer) == 1:
            payer_name = payer.index[0]
            total_owed_by_others = -payee.sum()
            actual_paid = row['Cost']
            row[payer_name] = -(actual_paid - total_owed_by_others)

        return row

split_df = split_df.apply(adjust_contribution, axis=1)

In [6]:
#split_df.to_csv("trial.csv", index=False)

## Bank details using Excel

In [7]:
bank_df = pd.read_excel(r"C:\Users\sanja\Documents\Projects\API\Simplii.xlsx")
bank_df

,Date,TRANSACTIONS,Funds out,Funds in,Running Balance
0,2025-08-18,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,164.02,Not applicable,19440.58
1,2025-08-18,POS MERCHANDISE TIM HORTONS 11,3.7,Not applicable,19604.60
2,2025-08-15,POS MERCHANDISE ESSO CIRCLE K,14.97,Not applicable,19608.30
3,2025-08-15,PAYROLL DEPOSIT THE ROYALE LP,Not applicable,1295.48,19623.27
4,2025-08-11,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,950,Not applicable,18327.79
5,2025-08-11,INTERAC E-TRANSFER SEND Aakash Georgian,30,Not applicable,19277.79
6,2025-08-11,POS MERCHANDISE 00055 MACS CONV,14.52,Not applicable,19307.79
7,2025-08-08,POS MERCHANDISE TIM HORTONS 11,1.92,Not applicable,19322.31
8,2025-08-08,POS PURCHASE GOOGLE Google One,3.15,Not applicable,19324.23
9,2025-08-08,EFT CREDIT CANADA,Not applicable,70.17,19327.38


In [8]:
creditbank_df = pd.read_excel(r"C:\Users\sanja\Documents\Projects\API\CreditCard.xlsx")
creditbank_df

,TRANSACTION DATE,DETAILS,AMOUNT
0,2025-08-02,Personal & Household ExpensesBELL MOBILITY VER...,−$51.99
1,2025-08-02,PAYMENT THANK YOU/PAIEMEN T MERCI,512.6
2,2025-07-31,"TransportationPRESTO MOBL TORONTO, ON",−$20.00
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,−$276.31


In [9]:
split_df 

,Date,Group,Description,Category,Cost,Currency,Aakash Raj,Preethi None,Aditya None,Rushda Najeeb,Beuju A,Deepak None,Aswin Narayanan,sai meghana,Praveen Kumar,Divyaa Saravanan,Vishnu Pratheep,Sanjay Kurian,Payment_or_not
0,2025-08-18,171 Anne Street,Sanjay Kurian paid Aswin Narayanan,General,164.02,CAD,0.0,0.0,0.0,0.00,0.0,0.0,164.02,0.0,0.0,0.0,0.0,-164.02,Payment
1,2025-08-12,171 Anne Street,Taste of shawarma,General,32.00,CAD,0.0,0.0,0.0,0.00,0.0,0.0,-16.00,0.0,0.0,0.0,0.0,-16.00,Taste of shawarma
2,2025-08-11,171 Anne Street,Nawabs uber,Taxi,40.00,CAD,0.0,0.0,0.0,0.00,0.0,0.0,-20.00,0.0,0.0,0.0,0.0,-20.00,Nawabs uber
3,2025-08-01,Us,Rushda Najeeb paid Sanjay Kurian,General,37.32,CAD,0.0,0.0,0.0,-37.32,0.0,0.0,0.00,0.0,0.0,0.0,0.0,37.32,Payment
4,2025-07-30,171 Anne Street,Cig,Entertainment - Other,14.00,CAD,0.0,0.0,0.0,0.00,0.0,0.0,-7.00,0.0,0.0,0.0,0.0,-7.00,Cig
5,2025-07-24,171 Anne Street,Cig,Entertainment - Other,18.53,CAD,0.0,0.0,0.0,0.00,0.0,0.0,-9.26,0.0,0.0,0.0,0.0,-9.27,Cig
6,2025-07-21,171 Anne Street,Cig,Entertainment - Other,42.00,CAD,0.0,0.0,0.0,0.00,0.0,0.0,-21.00,0.0,0.0,0.0,0.0,-21.00,Cig


In [10]:
#split_df.describe()

# Data Preprocessing

In [11]:
columns_to_keep = ["Date", "Description", "Category", "Cost", "Sanjay Kurian"]
split_df = split_df[columns_to_keep]

In [12]:
split_df

,Date,Description,Category,Cost,Sanjay Kurian
0,2025-08-18,Sanjay Kurian paid Aswin Narayanan,General,164.02,-164.02
1,2025-08-12,Taste of shawarma,General,32.00,-16.00
2,2025-08-11,Nawabs uber,Taxi,40.00,-20.00
3,2025-08-01,Rushda Najeeb paid Sanjay Kurian,General,37.32,37.32
4,2025-07-30,Cig,Entertainment - Other,14.00,-7.00
5,2025-07-24,Cig,Entertainment - Other,18.53,-9.27
6,2025-07-21,Cig,Entertainment - Other,42.00,-21.00


### Extracing all the expenses which involves Me.

In [13]:
split_df = split_df[split_df["Sanjay Kurian"] != 0]
split_df = split_df[split_df["Description"] != "Total balance"]
split_df

,Date,Description,Category,Cost,Sanjay Kurian
0,2025-08-18,Sanjay Kurian paid Aswin Narayanan,General,164.02,-164.02
1,2025-08-12,Taste of shawarma,General,32.00,-16.00
2,2025-08-11,Nawabs uber,Taxi,40.00,-20.00
3,2025-08-01,Rushda Najeeb paid Sanjay Kurian,General,37.32,37.32
4,2025-07-30,Cig,Entertainment - Other,14.00,-7.00
5,2025-07-24,Cig,Entertainment - Other,18.53,-9.27
6,2025-07-21,Cig,Entertainment - Other,42.00,-21.00


In [14]:
split_df.dtypes

Date              object
Description       object
Category          object
Cost             float64
Sanjay Kurian    float64
dtype: object

In [15]:
split_df["Cost"] = pd.to_numeric(split_df["Cost"], errors="coerce")

In [16]:
# mask = (split_df["Sanjay Kurian"] > 0) & (split_df["Sanjay Kurian"] == split_df["Cost"]) & (split_df["Category"] == "Payment")
# split_df.loc[mask, "Sanjay Kurian"] = split_df.loc[mask, "Sanjay Kurian"]

In [17]:
split_df

,Date,Description,Category,Cost,Sanjay Kurian
0,2025-08-18,Sanjay Kurian paid Aswin Narayanan,General,164.02,-164.02
1,2025-08-12,Taste of shawarma,General,32.00,-16.00
2,2025-08-11,Nawabs uber,Taxi,40.00,-20.00
3,2025-08-01,Rushda Najeeb paid Sanjay Kurian,General,37.32,37.32
4,2025-07-30,Cig,Entertainment - Other,14.00,-7.00
5,2025-07-24,Cig,Entertainment - Other,18.53,-9.27
6,2025-07-21,Cig,Entertainment - Other,42.00,-21.00


In [18]:
# unique_descriptions = split_df["Description"].value_counts()
# unique_descriptions

In [19]:
# known_descriptions = ["Walmart", "Costco", "Uber", "Centra", "Dollarama", "Wifi", "LCBO", "Subway"]

# split_df["Description_clean"] = split_df["Description"].str.strip()

# # Get all unique descriptions
# unique_descriptions = split_df["Description_clean"].unique()

# # Create an empty mapping dictionary
# description_map = {}

# for desc in unique_descriptions:
#     if desc in known_descriptions:
#         description_map[desc] = desc  # keep as-is
#     else:
#         corrected = input(f"Enter a standardized name for: '{desc}' → ")
#         description_map[desc] = corrected

# # Apply the mapping to create a normalized column
# split_df["Description_normalized"] = split_df["Description_clean"].map(description_map)

In [20]:
#split_df

In [21]:
# unique_descriptions = split_df["Description_normalized"].value_counts()
# unique_descriptions

In [22]:
# split_df[split_df["Description_normalized"] == ""]

In [23]:
# split_df["Description_normalized"] = split_df["Description_normalized"].fillna("Walmart")

In [24]:
#split_df.to_excel("Splitwise_Cleaned.xlsx", index=False)

In [25]:
bank_df

,Date,TRANSACTIONS,Funds out,Funds in,Running Balance
0,2025-08-18,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,164.02,Not applicable,19440.58
1,2025-08-18,POS MERCHANDISE TIM HORTONS 11,3.7,Not applicable,19604.60
2,2025-08-15,POS MERCHANDISE ESSO CIRCLE K,14.97,Not applicable,19608.30
3,2025-08-15,PAYROLL DEPOSIT THE ROYALE LP,Not applicable,1295.48,19623.27
4,2025-08-11,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,950,Not applicable,18327.79
5,2025-08-11,INTERAC E-TRANSFER SEND Aakash Georgian,30,Not applicable,19277.79
6,2025-08-11,POS MERCHANDISE 00055 MACS CONV,14.52,Not applicable,19307.79
7,2025-08-08,POS MERCHANDISE TIM HORTONS 11,1.92,Not applicable,19322.31
8,2025-08-08,POS PURCHASE GOOGLE Google One,3.15,Not applicable,19324.23
9,2025-08-08,EFT CREDIT CANADA,Not applicable,70.17,19327.38


* Replacing Not applicable to 0
* Negating funds out to show outward cash flow
* Combining Funds in and Funds out as Funds

In [26]:
bank_df[['Funds out', 'Funds in']] = bank_df[['Funds out', 'Funds in']].replace('Not applicable', 0)
bank_df["Funds out"] = -bank_df["Funds out"]
bank_df["Funds out"] = pd.to_numeric(bank_df["Funds out"])
bank_df["Funds in"] = pd.to_numeric(bank_df["Funds in"])
bank_df["Funds"] = bank_df["Funds in"] + bank_df["Funds out"]

C:\Users\sanja\AppData\Local\Temp\ipykernel_38872\3164022823.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bank_df[['Funds out', 'Funds in']] = bank_df[['Funds out', 'Funds in']].replace('Not applicable', 0)


In [27]:
bank_df

,Date,TRANSACTIONS,Funds out,Funds in,Running Balance,Funds
0,2025-08-18,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,-164.02,0.00,19440.58,-164.02
1,2025-08-18,POS MERCHANDISE TIM HORTONS 11,-3.70,0.00,19604.60,-3.70
2,2025-08-15,POS MERCHANDISE ESSO CIRCLE K,-14.97,0.00,19608.30,-14.97
3,2025-08-15,PAYROLL DEPOSIT THE ROYALE LP,-0.00,1295.48,19623.27,1295.48
4,2025-08-11,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,-950.00,0.00,18327.79,-950.00
5,2025-08-11,INTERAC E-TRANSFER SEND Aakash Georgian,-30.00,0.00,19277.79,-30.00
6,2025-08-11,POS MERCHANDISE 00055 MACS CONV,-14.52,0.00,19307.79,-14.52
7,2025-08-08,POS MERCHANDISE TIM HORTONS 11,-1.92,0.00,19322.31,-1.92
8,2025-08-08,POS PURCHASE GOOGLE Google One,-3.15,0.00,19324.23,-3.15
9,2025-08-08,EFT CREDIT CANADA,-0.00,70.17,19327.38,70.17


In [28]:
bank_df.drop(columns=["Funds in", "Funds out"], inplace=True)

In [29]:
bank_df = bank_df.dropna(subset=["Date"])

In [30]:
bank_df

,Date,TRANSACTIONS,Running Balance,Funds
0,2025-08-18,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,19440.58,-164.02
1,2025-08-18,POS MERCHANDISE TIM HORTONS 11,19604.60,-3.70
2,2025-08-15,POS MERCHANDISE ESSO CIRCLE K,19608.30,-14.97
3,2025-08-15,PAYROLL DEPOSIT THE ROYALE LP,19623.27,1295.48
4,2025-08-11,INTERAC E-TRANSFER SEND Aswin Narayanan London Pr,18327.79,-950.00
5,2025-08-11,INTERAC E-TRANSFER SEND Aakash Georgian,19277.79,-30.00
6,2025-08-11,POS MERCHANDISE 00055 MACS CONV,19307.79,-14.52
7,2025-08-08,POS MERCHANDISE TIM HORTONS 11,19322.31,-1.92
8,2025-08-08,POS PURCHASE GOOGLE Google One,19324.23,-3.15
9,2025-08-08,EFT CREDIT CANADA,19327.38,70.17


### Replacing all the common transactions with common terms

In [31]:
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("INTERAC E-TRANSFER ", "", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("POS MERCHANDISE ", "", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("POS PURCHASE ", "", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("PAYROLL DEPOSIT THE ROYALE LP", "Salary", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("EFT CREDIT CANADA", "Tax Returns", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("EFT CREDIT TPS/GST", "Tax Returns", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("ABM WITHDRAWAL", "ATM Withdrawal", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("ABM DEPOSIT", "ATM Deposit", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("EFT CREDIT ISBO ReferralCR", "Referral", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("REMISE CARBONE/CARBON REBATE", "Tax Returns", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("EFT CREDIT NO FEE CASH REWARD", "Referral", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("INTERNET BILL PAYMENT VISA, CIBC/BANQUE CIBC", "Credit Card Bill", regex=False)
bank_df["TRANSACTIONS"] = bank_df["TRANSACTIONS"].str.replace("TRANSFER CREDIT TO CURRENT REFERRAL BONUS231768381", "Referral", regex=False)

In [32]:
bank_df.drop("Running Balance", axis=1, inplace=True)

In [33]:
bank_df

,Date,TRANSACTIONS,Funds
0,2025-08-18,SEND Aswin Narayanan London Pr,-164.02
1,2025-08-18,TIM HORTONS 11,-3.70
2,2025-08-15,ESSO CIRCLE K,-14.97
3,2025-08-15,Salary,1295.48
4,2025-08-11,SEND Aswin Narayanan London Pr,-950.00
5,2025-08-11,SEND Aakash Georgian,-30.00
6,2025-08-11,00055 MACS CONV,-14.52
7,2025-08-08,TIM HORTONS 11,-1.92
8,2025-08-08,GOOGLE Google One,-3.15
9,2025-08-08,Tax Returns,70.17


In [34]:
creditbank_df

,TRANSACTION DATE,DETAILS,AMOUNT
0,2025-08-02,Personal & Household ExpensesBELL MOBILITY VER...,−$51.99
1,2025-08-02,PAYMENT THANK YOU/PAIEMEN T MERCI,512.6
2,2025-07-31,"TransportationPRESTO MOBL TORONTO, ON",−$20.00
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,−$276.31


In [35]:
creditbank_df["AMOUNT"] = (
    creditbank_df["AMOUNT"]
    .astype(str)
    .str.replace("\u2212", "-", regex=False)
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)
creditbank_df["AMOUNT"] = pd.to_numeric(creditbank_df["AMOUNT"], errors="coerce")

In [36]:
creditbank_df

,TRANSACTION DATE,DETAILS,AMOUNT
0,2025-08-02,Personal & Household ExpensesBELL MOBILITY VER...,-51.99
1,2025-08-02,PAYMENT THANK YOU/PAIEMEN T MERCI,512.60
2,2025-07-31,"TransportationPRESTO MOBL TORONTO, ON",-20.00
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31


In [37]:
creditbank_df["DETAILS"] = creditbank_df["DETAILS"].str.replace("Personal & Household ExpensesBELL MOBILITY VERDUN, QC", "Phone data", regex=False)
creditbank_df = creditbank_df.rename(columns={
    "TRANSACTION DATE": "Date",
    "DETAILS": "TRANSACTIONS",
    "AMOUNT": "Funds"
})
creditbank_df["Account"] = "CIBC Credit Card"
creditbank_df

,Date,TRANSACTIONS,Funds,Account
0,2025-08-02,Phone data,-51.99,CIBC Credit Card
1,2025-08-02,PAYMENT THANK YOU/PAIEMEN T MERCI,512.60,CIBC Credit Card
2,2025-07-31,"TransportationPRESTO MOBL TORONTO, ON",-20.00,CIBC Credit Card
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31,CIBC Credit Card


In [38]:
#split_df.drop(columns=["Description","Category","Cost","Description_clean"])
split_df.drop(columns=["Category","Cost"])

,Date,Description,Sanjay Kurian
0,2025-08-18,Sanjay Kurian paid Aswin Narayanan,-164.02
1,2025-08-12,Taste of shawarma,-16.00
2,2025-08-11,Nawabs uber,-20.00
3,2025-08-01,Rushda Najeeb paid Sanjay Kurian,37.32
4,2025-07-30,Cig,-7.00
5,2025-07-24,Cig,-9.27
6,2025-07-21,Cig,-21.00


In [39]:
split_df = split_df[["Date", "Description", "Sanjay Kurian"]]
split_df = split_df.rename(columns={
    "Description": "TRANSACTIONS",
    "Sanjay Kurian": "Funds"
})

In [40]:
split_df

,Date,TRANSACTIONS,Funds
0,2025-08-18,Sanjay Kurian paid Aswin Narayanan,-164.02
1,2025-08-12,Taste of shawarma,-16.00
2,2025-08-11,Nawabs uber,-20.00
3,2025-08-01,Rushda Najeeb paid Sanjay Kurian,37.32
4,2025-07-30,Cig,-7.00
5,2025-07-24,Cig,-9.27
6,2025-07-21,Cig,-21.00


In [41]:
split_df["Date"] = pd.to_datetime(split_df["Date"]).dt.date
bank_df["Date"] = pd.to_datetime(bank_df["Date"]).dt.date
creditbank_df["Date"] = pd.to_datetime(creditbank_df["Date"]).dt.date
split_df["Account"] = "Chequing"
bank_df["Account"] = "Chequing"
combined_df = pd.concat([split_df, bank_df, creditbank_df])
combined_df = combined_df.sort_values(by="Date").reset_index(drop=True)
combined_df

,Date,TRANSACTIONS,Funds,Account
0,2025-07-18,Salary,1306.56,Chequing
1,2025-07-21,Cig,-21.00,Chequing
2,2025-07-23,00055 MACS CONV,-18.53,Chequing
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31,CIBC Credit Card
4,2025-07-24,Cig,-9.27,Chequing
5,2025-07-28,UBER CANADA/UBEREATS,-28.29,Chequing
6,2025-07-30,BOMBAY SPICES,-4.88,Chequing
7,2025-07-30,Cig,-7.00,Chequing
8,2025-07-31,TIM HORTONS 12,-3.80,Chequing
9,2025-07-31,DOWNSVIEW PARK,-20.00,Chequing


In [42]:
#combined_df.to_excel("Combined_Cleaned.xlsx", index=False)

### Splitwise has records of payment and the exact amount is recorded in the bank transactions. I deleted both the records or else expenses may get duplicated

In [43]:
combined_df["Name"] = combined_df["TRANSACTIONS"].str.extract(r'(?:SEND|paid|RECEIVE)\s+([A-Za-z]+)', expand=False)
combined_df.loc[combined_df["Name"] == 'Sanjay', "Name"] = combined_df["TRANSACTIONS"].str.extract(r'^(\w+)', expand=False)
combined_df["Name"] = combined_df["Name"].str.capitalize()
combined_df["AbsFunds"] = combined_df["Funds"]

In [44]:
combined_df

,Date,TRANSACTIONS,Funds,Account,Name,AbsFunds
0,2025-07-18,Salary,1306.56,Chequing,NaN,1306.56
1,2025-07-21,Cig,-21.00,Chequing,NaN,-21.00
2,2025-07-23,00055 MACS CONV,-18.53,Chequing,NaN,-18.53
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31,CIBC Credit Card,NaN,-276.31
4,2025-07-24,Cig,-9.27,Chequing,NaN,-9.27
5,2025-07-28,UBER CANADA/UBEREATS,-28.29,Chequing,NaN,-28.29
6,2025-07-30,BOMBAY SPICES,-4.88,Chequing,NaN,-4.88
7,2025-07-30,Cig,-7.00,Chequing,NaN,-7.00
8,2025-07-31,TIM HORTONS 12,-3.80,Chequing,NaN,-3.80
9,2025-07-31,DOWNSVIEW PARK,-20.00,Chequing,NaN,-20.00


In [45]:
mask_paid = combined_df["TRANSACTIONS"].str.contains("paid", case=False)
mask_receive = combined_df["TRANSACTIONS"].str.contains("RECEIVE", case=False)
mask_send = combined_df["TRANSACTIONS"].str.contains("SEND", case=False)

paid_trans = combined_df[mask_paid]
receive_trans = combined_df[mask_receive]
send_trans = combined_df[mask_send]

to_delete_indices = []

for _, paid_row in paid_trans.iterrows():
    matching_receives = receive_trans[
        (receive_trans["Name"] == paid_row["Name"]) & 
        (receive_trans["AbsFunds"] == paid_row["AbsFunds"])
    ]

    if not matching_receives.empty:
        to_delete_indices.extend([paid_row.name, matching_receives.index[0]])
        receive_trans = receive_trans.drop(matching_receives.index[0])

for _, paid_row in paid_trans.iterrows():
    matching_sends = send_trans[
        (send_trans["Name"] == paid_row["Name"]) & 
        (send_trans["AbsFunds"] == paid_row["AbsFunds"])
    ]
    
    if not matching_sends.empty:
        to_delete_indices.extend([paid_row.name, matching_sends.index[0]])
        send_trans = send_trans.drop(matching_sends.index[0])
to_delete_indices = list(set(to_delete_indices))
cleaned_df = combined_df[~combined_df.index.isin(to_delete_indices)]

In [46]:
cleaned_df = cleaned_df.drop(columns=["Name", "AbsFunds"])
cleaned_df = cleaned_df[~cleaned_df["TRANSACTIONS"].str.contains("paid", case=False, na=False)]
cleaned_df

,Date,TRANSACTIONS,Funds,Account
0,2025-07-18,Salary,1306.56,Chequing
1,2025-07-21,Cig,-21.00,Chequing
2,2025-07-23,00055 MACS CONV,-18.53,Chequing
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31,CIBC Credit Card
4,2025-07-24,Cig,-9.27,Chequing
5,2025-07-28,UBER CANADA/UBEREATS,-28.29,Chequing
6,2025-07-30,BOMBAY SPICES,-4.88,Chequing
7,2025-07-30,Cig,-7.00,Chequing
8,2025-07-31,TIM HORTONS 12,-3.80,Chequing
9,2025-07-31,DOWNSVIEW PARK,-20.00,Chequing


In [47]:
#cleaned_df.to_excel("Combined_Cleaned_Transactions.xlsx", index=False)

In [48]:
notes_df = cleaned_df[["TRANSACTIONS"]]
notes_df

,TRANSACTIONS
0,Salary
1,Cig
2,00055 MACS CONV
3,Foreign Currency TransactionsWL *VUE*Testing E...
4,Cig
5,UBER CANADA/UBEREATS
6,BOMBAY SPICES
7,Cig
8,TIM HORTONS 12
9,DOWNSVIEW PARK


### Importing an excel file which contains all the common categories for expenses. Easier to categorize the expenses.

In [49]:
reference_df = pd.read_excel(r"C:\Users\sanja\Documents\Projects\API\Categories.xlsx")

In [50]:
reference_df

,Notes,Category
0,Bus card,Transportation
1,Uber,Transportation
2,Cash withdrawal,Other
3,Salary,Salary
4,Pizza pizza,Food
...,...,...
183,India Funds,Petty cash
184,UAE Visa,Visa
185,Consultancy,Service charge
186,Sheesha,Social Life


### Using Rapid fuzz to convert all the typos in the splitwise to the correct transaction and mapping to appropriate category.

In [51]:
from rapidfuzz import process

correct_names = reference_df['Notes'].tolist()

# Function to apply fuzzy matching
def match_transaction(txn):
    best_match, score, _ = process.extractOne(txn, correct_names)
    if score >= 90:
        category = reference_df.loc[reference_df['Notes'] == best_match, 'Category'].values[0]
        return pd.Series([best_match, category, score])
    else:
        return pd.Series([None, None, score])

# Apply the matching function to your messy list of notes
cleaned_df[['Matched_Name', 'Category', 'Score']] = cleaned_df['TRANSACTIONS'].apply(match_transaction)

In [52]:
cleaned_df

,Date,TRANSACTIONS,Funds,Account,Matched_Name,Category,Score
0,2025-07-18,Salary,1306.56,Chequing,Salary,Salary,100.000000
1,2025-07-21,Cig,-21.00,Chequing,NaN,NaN,72.000000
2,2025-07-23,00055 MACS CONV,-18.53,Chequing,NaN,NaN,60.000000
3,2025-07-24,Foreign Currency TransactionsWL *VUE*Testing E...,-276.31,CIBC Credit Card,NaN,NaN,60.000000
4,2025-07-24,Cig,-9.27,Chequing,NaN,NaN,72.000000
5,2025-07-28,UBER CANADA/UBEREATS,-28.29,Chequing,NaN,NaN,60.000000
6,2025-07-30,BOMBAY SPICES,-4.88,Chequing,NaN,NaN,60.000000
7,2025-07-30,Cig,-7.00,Chequing,NaN,NaN,72.000000
8,2025-07-31,TIM HORTONS 12,-3.80,Chequing,NaN,NaN,60.000000
9,2025-07-31,DOWNSVIEW PARK,-20.00,Chequing,NaN,NaN,60.000000


In [53]:
cleaned_df.loc[cleaned_df['Matched_Name'].notna(), 'TRANSACTIONS'] = cleaned_df['Matched_Name']
cleaned_df.drop(columns=['Matched_Name', 'Score'], inplace=True)
cleaned_df = cleaned_df[['Date', 'Account', 'TRANSACTIONS', 'Category', 'Funds']]

In [54]:
cleaned_df

,Date,Account,TRANSACTIONS,Category,Funds
0,2025-07-18,Chequing,Salary,Salary,1306.56
1,2025-07-21,Chequing,Cig,NaN,-21.00
2,2025-07-23,Chequing,00055 MACS CONV,NaN,-18.53
3,2025-07-24,CIBC Credit Card,Foreign Currency TransactionsWL *VUE*Testing E...,NaN,-276.31
4,2025-07-24,Chequing,Cig,NaN,-9.27
5,2025-07-28,Chequing,UBER CANADA/UBEREATS,NaN,-28.29
6,2025-07-30,Chequing,BOMBAY SPICES,NaN,-4.88
7,2025-07-30,Chequing,Cig,NaN,-7.00
8,2025-07-31,Chequing,TIM HORTONS 12,NaN,-3.80
9,2025-07-31,Chequing,DOWNSVIEW PARK,NaN,-20.00


### Adding Running balance

In [55]:
cleaned_df["Running Balance"] = cleaned_df["Funds"].cumsum() + last_updated_balance

C:\Users\sanja\AppData\Local\Temp\ipykernel_38872\1658279965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["Running Balance"] = cleaned_df["Funds"].cumsum() + last_updated_balance


In [56]:
latest_balance = cleaned_df['Running Balance'].iloc[-1]

In [57]:
cleaned_df["Income/Expense"] = cleaned_df["Funds"].apply(lambda x: "Expense" if x < 0 else "Income")
cleaned_df = cleaned_df.rename(columns= {
    "TRANSACTIONS": "Note",
    "Funds": "Amount"
})

C:\Users\sanja\AppData\Local\Temp\ipykernel_38872\3320630998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["Income/Expense"] = cleaned_df["Funds"].apply(lambda x: "Expense" if x < 0 else "Income")


In [58]:
cleaned_df = cleaned_df[["Date", "Account", "Category", "Note", "Income/Expense", "Amount"]]
cleaned_df

,Date,Account,Category,Note,Income/Expense,Amount
0,2025-07-18,Chequing,Salary,Salary,Income,1306.56
1,2025-07-21,Chequing,NaN,Cig,Expense,-21.00
2,2025-07-23,Chequing,NaN,00055 MACS CONV,Expense,-18.53
3,2025-07-24,CIBC Credit Card,NaN,Foreign Currency TransactionsWL *VUE*Testing E...,Expense,-276.31
4,2025-07-24,Chequing,NaN,Cig,Expense,-9.27
5,2025-07-28,Chequing,NaN,UBER CANADA/UBEREATS,Expense,-28.29
6,2025-07-30,Chequing,NaN,BOMBAY SPICES,Expense,-4.88
7,2025-07-30,Chequing,NaN,Cig,Expense,-7.00
8,2025-07-31,Chequing,NaN,TIM HORTONS 12,Expense,-3.80
9,2025-07-31,Chequing,NaN,DOWNSVIEW PARK,Expense,-20.00


# Data Export

In [59]:
cleaned_df.to_excel("Final.xlsx", index=False)

In [60]:
# from openpyxl import load_workbook
# from openpyxl.utils.dataframe import dataframe_to_rows

# cleaned_df = pd.read_excel(r"C:\Users\sanja\Documents\Projects\API\Final.xlsx")
# filename = r'C:\Users\sanja\Documents\Projects\API\MM.xlsx'
# sheet_name = 'Sheet1' 
# table_name = 'MyTable'  

# wb = load_workbook(filename)
# ws = wb[sheet_name]


# table = ws.tables[table_name]
# table_range = table.ref 
# start_cell, end_cell = table_range.split(':')
# last_row = ws[end_cell].row

# for row in dataframe_to_rows(cleaned_df, index=False, header=False):
#     ws.append(row)

# new_end_row = last_row + len(cleaned_df)
# table.ref = f"{start_cell.split(':')[0]}:{end_cell[:1]}{new_end_row}"

# wb.save(filename)

### Saving the Latest date and latest balance to text file

In [61]:
# latest_date = cleaned_df['Date'].max()

# with open('last_updated.txt', 'w') as f:
#     f.write(latest_date.isoformat() + '\n')
#     #f.write(str(latest_balance))